## Step 1: Using SeqIO, read in and parse the file of apple primary transcripts (available on Canvas):
    - During your parsing, create a list of only transcripts of length 125 or less
    - Report the number of transcripts matching this criteria
    - Translate these sequences to protein, make sure to save them to their own list

In [1]:
from Bio import SeqIO

seqList = []
translatedSeq = []
for seq_record in SeqIO.parse("Mdomestica_491_v1.1.cds_primaryTranscriptOnly.fa", "fasta"):
        if len(seq_record) <= 125:
            seqList.append(seq_record)
            translatedSeq.append(seq_record.translate(to_stop=True)) # http://biopython.org/DIST/docs/tutorial/Tutorial.html 3.8 Translation

# print(translatedSeq)
print("transcripts of length 125 or less: "+ str(len(seqList)))

transcripts of length 125 or less: 62


## Step 2: Align our small proteins using pairwise2
    - Align each protein to each other protein using the blosum62 scoring matrix, a gap opening penalty of -10, and a gap extension penalty of -0.5
    - If an alignment scores better than 40, save it.
    - Print the highest scoring pairwise alignment
    - Take care in how you set up your loops for the pairwise alignments. Each alignment only needs to be done once. If you've already tested the alignment of seq1 vs seq5, don't align seq5 vs seq1.
    

In [2]:
from Bio import pairwise2
from Bio.Align import substitution_matrices
blosum62 = substitution_matrices.load("BLOSUM62")

alignments = []
alignmentScore = 40
highestScoringPairwiseAlignment = ""

i=0
while i in range(len(translatedSeq)):
    if i%2 > 0: #skip to everyother one for processing improvment
        alignments.append(pairwise2.align.globalds(translatedSeq[i-1].seq, translatedSeq[i].seq,blosum62, -10, -0.5))            
    i+=1
# print(alignments)
    
for elements in alignments:
#     print(elements)
    if elements[0][4] < 40: # from above activity we know that score is present in the 4th index
        alignments.remove(elements)
    else:
        if elements[0][4] >= alignmentScore: # update everytime we get a new score more than the current Alignment Score
            alignmentScore = elements[0][4]
            highestScoringPairwiseAlignment = elements # Assign the pair wise alignment with max score
            
print(str(alignmentScore) + " is the highest score!")
print("Print the highest scoring pairwise alignment :", highestScoringPairwiseAlignment)

64 is the highest score!
Print the highest scoring pairwise alignment : [Alignment(seqA='MI------------------WF------CFVFLGNGVVRFSQLLGCHFHVGSTSTKLRVRICKI', seqB='MLSDGEPVPCLHRSCGSKKSWFFKASFCCF-----------ELLAFFYP-------------KL', score=-8.0, start=0, end=64)]


## Step 3: Running BLAST and reading results
     *Because we are doing a web BLAST, choose ONLY one sequence from our list of short proteins*.
    - Run BLAST with your sequence against the NR database
    - Parse the results. Report any HSPs with an E-value less than 0.05 and show the HSP alignments, including the name of the matching sequence. If no HSPs meet that criteria, report the highest scoring pair.
    - If for whatever reason the sequence you selected fails to return any results, try a new one 

In [3]:
from Bio.Blast import NCBIXML, NCBIWWW
Seq = translatedSeq[2] # sequence are present in the second index

result_handle = NCBIWWW.qblast("blastp", "nr", Seq.seq)

with open('my_blast.xml', 'w') as save_file: 
    blast_record = result_handle.read() 
    save_file.write(blast_record)

In [4]:
E_VALUE_THRESH = 0.05
for blast_record in NCBIXML.parse(open("my_blast.xml")): 
    if blast_record.alignments: 
        print("****Alignment****")
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                if hsp.expect < E_VALUE_THRESH: 
                    print("Sequence ", alignment.title)
                else:
                    print("Sequence Not met the threshold :", alignment.title)

****Alignment****
Sequence  gb|TQD69456.1| hypothetical protein C1H46_045011 [Malus baccata]
Sequence  gb|TQD97470.1| hypothetical protein C1H46_016931 [Malus baccata]
Sequence  gb|TQE13584.1| hypothetical protein C1H46_000591 [Malus baccata]
Sequence  gb|KAB2596638.1| hypothetical protein D8674_032088 [Pyrus ussuriensis x Pyrus communis]
Sequence  gb|RXH99232.1| hypothetical protein DVH24_011557 [Malus domestica]
Sequence  gb|TQD92717.1| hypothetical protein C1H46_021710 [Malus baccata]
Sequence  gb|KAB2610802.1| hypothetical protein D8674_018834 [Pyrus ussuriensis x Pyrus communis]
Sequence  gb|KAB2616017.1| hypothetical protein D8674_022605 [Pyrus ussuriensis x Pyrus communis]
Sequence  gb|KAB2606699.1| hypothetical protein D8674_006416 [Pyrus ussuriensis x Pyrus communis]
Sequence Not met the threshold : ref|WP_191618210.1| PQQ-dependent sugar dehydrogenase [Pelagicoccus sp. NFK12] >gb|MBD5781106.1| PQQ-dependent sugar dehydrogenase [Pelagicoccus sp. NFK12]
